# Write images to videos

Conclusion: use `ffmpeg`.

I initially tried `opencv-python` but had trouble with large videos. Along the way I learned that `opencv-python` are not designed for batch processing, so the file size has a cap of 2 GB. Instead, `ffmpeg` is a much more powerful tool in batch processing: faster and more stable.

To use `ffmpeg` more efficiently, I also learned to invoke CLI commands from python script. This can make scripts more intelligent when search the right directories to perform the command.

The code below are some tests during the course of learning.

In [1]:
import cv2
from skimage import io
import os
from corrLib import readdata
from myImageLib import to8bit
import time

In [21]:
test_image_folder = r'C:\Users\liuzy\Documents\Data\DE\nov22\00'
l = readdata(test_image_folder, 'tif')

In [22]:
%%time
sample_img = io.imread(l.Dir[0])
video = cv2.VideoWriter(os.path.join(test_image_folder, 'video.avi'), cv2.VideoWriter_fourcc(*'DIVX'), 50, sample_img.shape, isColor=False)
for num, i in l.iterrows():
    img = io.imread(i.Dir)
    video.write(to8bit(img))
video.release()

Wall time: 16.2 s


## Invoke `ffmpeg`

In [23]:
os.system('dir')

0

When the python script is run, the `dir` command will be invoked. SIMPLE!

The following command directly convert images to mp4, bypassing the need for opencv
```
ffmpeg -f image2 -r 1/5 -i ./images/swissGenevaLake%01d.jpg -vcodec mpeg4 -y ./videos/swissGenevaLake.mp4
```

In [24]:
os.system('ffmpeg -f image2 -r 50 -i {0}\*.tif -vcodec mpeg4 -y {1}.mp4'.format(test_image_folder, test_image_folder))

1

## Incompatible with 16-bit gray

When converting 16-bit gray images to videos, `ffmpeg` does not adjust the contrast and the videos look very bad. In this section, we try to use `colorlevels` filter to manually adjust the contrast, to make the videos look better. [[ref]](https://stackoverflow.com/questions/66155414/convert-16bit-grayscale-png-to-hevc-x265)

Though 16-bit gray images don't work well when converting to videos, the 8-bit images wors very well. 

So far `h264` is the best encoder I find. An avi output will be produced for each image sequence. The output video is quite small in size. For instance, a 12 GB nd2 video will be converted to a ~70 MB avi video. The compression ratio is around 171, the following command can do the conversion:
```
ffmpeg -framerate xx -i input -vcodec h264 output.avi
```

In [26]:
folder = r'C:\Users\liuzy\Documents\Data\DE\nov22'

In [41]:
l = dirrec(folder, '00000.tif')

In [29]:
dirrec

<function myImageLib.dirrec(path, filename)>

In [39]:
input_folder = os.path.join(folder, '00', '8-bit')
output_file = os.path.join(folder, '00.avi')
# input format: by default it's %05d, however custom format should be supported by passing keyword fmt
# framerate: by default frame rate is set to 50, and it can be modified by passing keyword fps
fmt = "%05d.tif"
input_imseq = os.path.join(input_folder, fmt)
fps = 50
cmd = 'ffmpeg -framerate {0:d} -i {1} -vodec h264 {2}'.format(fps, input_imseq, output_file)
os.system(cmd)

1

In [43]:
for i in l:
    if "8-bit" in i:
        print(i)
        print(os.path.split(os.path.split(os.path.split(i)[0])[0])[1])

C:\Users\liuzy\Documents\Data\DE\nov22\00\8-bit\00000.tif
00
C:\Users\liuzy\Documents\Data\DE\nov22\02\8-bit\00000.tif
02
C:\Users\liuzy\Documents\Data\DE\nov22\03\8-bit\00000.tif
03
C:\Users\liuzy\Documents\Data\DE\nov22\04\8-bit\00000.tif
04
C:\Users\liuzy\Documents\Data\DE\nov22\05\8-bit\00000.tif
05
C:\Users\liuzy\Documents\Data\DE\nov22\06\8-bit\00000.tif
06
C:\Users\liuzy\Documents\Data\DE\nov22\07\8-bit\00000.tif
07
C:\Users\liuzy\Documents\Data\DE\nov22\08\8-bit\00000.tif
08
C:\Users\liuzy\Documents\Data\DE\nov22\09\8-bit\00000.tif
09
C:\Users\liuzy\Documents\Data\DE\nov22\10\8-bit\00000.tif
10
C:\Users\liuzy\Documents\Data\DE\nov22\count\00\8-bit\00000.tif
00
C:\Users\liuzy\Documents\Data\DE\nov22\count\00001\8-bit\00000.tif
00001
C:\Users\liuzy\Documents\Data\DE\nov22\count\00002\8-bit\00000.tif
00002
C:\Users\liuzy\Documents\Data\DE\nov22\count\00003\8-bit\00000.tif
00003
C:\Users\liuzy\Documents\Data\DE\nov22\count\00004\8-bit\00000.tif
00004
C:\Users\liuzy\Documents\Data\D

## Example of one folder processing

In [2]:
folder = '/media/zhengyang/NothingToSay/DE/10262021'
l = readdata(folder, 'nd2')
l.head()

,Name,Dir
0,00,/media/zhengyang/NothingToSay/DE/10262021/00.nd2
1,01,/media/zhengyang/NothingToSay/DE/10262021/01.nd2
2,02,/media/zhengyang/NothingToSay/DE/10262021/02.nd2
3,03,/media/zhengyang/NothingToSay/DE/10262021/03.nd2
4,04,/media/zhengyang/NothingToSay/DE/10262021/04.nd2


In [3]:
for num, i in l.iterrows():
    cmd = "python /home/zhengyang/Documents/GitHub/Python/generic_proc/py_files/to_tif.py {}".format(i.Dir)
    flag = os.system(cmd)
    print(flag)

0


## Add capacity check function to to_tif

Since to_tif generates a big volume of images, there is a risk that the disk storage runs out. To avoid this potential issue, I add a capacity check function to the to_tif.py script. If the available disk space is no more than twice of the nd2 file being converted, abort the conversion. 

In [7]:
file_size = os.path.getsize(i.Dir)
file_size

1048735744

In [12]:
import shutil

disk_dir = "/media/zhengyang/NothingToSay/DE/10262021/00.nd2"
total, used, free = shutil.disk_usage(disk_dir)
free

1218070913024

In [13]:
free / 2**30

1134.4169387817383

In [8]:
free > 2 * file_size

True

True means it's good to proceed with the conversion!